In [ ]:
pip install pykafka


In [ ]:
pip install fastavro

In [ ]:
pip install confluent_kafka

In [42]:
from pyspark.sql import SparkSession
from pyspark.sql.avro.functions import from_avro
from pyspark.sql.functions import to_binary
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType
import json
from pykafka import KafkaClient
import pyspark.sql.functions as psf
from pyspark.sql.types import *
import io
import fastavro
from confluent_kafka.schema_registry import SchemaRegistryClient

In [ ]:
client = KafkaClient(hosts="broker:29092")

In [ ]:
kafka_topic = "PurchasedItems"

# Kafka broker adresi
kafka_bootstrap_servers = "broker:29092"

In [47]:
schemaRegistryAddr = "http://schema-registry:8081"

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,org.apache.spark:spark-avro_2.12:3.2.0') \
    .config("spark.sql.shuffle.partitions", 4) \
    .master("local").getOrCreate()

In [43]:
def get_schema_from_schema_registry(schema_registry_url, schema_registry_subject):
    sr = SchemaRegistryClient({'url': schema_registry_url})
    latest_version = sr.get_latest_version(schema_registry_subject)

    return sr, latest_version

In [48]:
schema, version = get_schema_from_schema_registry(schemaRegistryAddr, "PurchasedItems-value")

In [52]:
version.schema.schema_str

'{"type":"record","name":"PurchasedItemsSchema","fields":[{"name":"sessionId","type":"int"},{"name":"timestamp","type":"string"},{"name":"userId","type":"int"},{"name":"totalPrice","type":"double"},{"name":"orderId","type":"int"},{"name":"products","type":{"type":"array","items":{"type":"record","name":"products","namespace":"Record","fields":[{"name":"product_id","type":"int"},{"name":"item_count","type":"int"},{"name":"item_price","type":"double"},{"name":"item_discount","type":"double"}]}}}]}'

In [64]:
streaming_df = spark.readStream\
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
.option("mode","PERMISSIVE") \
    .load() \
    .select(from_avro("value", version.schema.schema_str).alias("value")).select("value.*")

In [65]:
streaming_df.printSchema()

root
 |-- sessionId: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- userId: integer (nullable = true)
 |-- totalPrice: double (nullable = true)
 |-- orderId: integer (nullable = true)
 |-- products: array (nullable = true)
 |    |-- element: struct (containsNull = false)
 |    |    |-- product_id: integer (nullable = false)
 |    |    |-- item_count: integer (nullable = false)
 |    |    |-- item_price: double (nullable = false)
 |    |    |-- item_discount: double (nullable = false)



In [ ]:
avro_struct = StructType([
    StructField("sessionId", IntegerType()),
    StructField("timestamp", StringType()),
    StructField("userId", IntegerType()),
    StructField("totalPrice", DoubleType()),
    StructField("orderId", IntegerType()) 
])

In [ ]:
avro_struct

In [ ]:
streaming_df = streaming_df.select(psf.expr("substring(value,1,1)"))

In [ ]:
jsonFormatSchema = open("PurchasedItemsSchema.avsc", "r").read()

In [ ]:
output = streaming_df\
  .select(from_avro(psf.col("value"), jsonFormatSchema).alias("avro")).select(psf.col("avro.*"))

In [ ]:
streaming_df.printSchema()

In [ ]:
output.printSchema()

In [68]:
query = streaming_df.writeStream \
    .outputMode("append") \
    .format("json") \
    .option("path", "purchased_items.json") \
    .option("checkpointLocation", "checkpoint-2") \
    .start()

# İşlemi başlat


In [ ]:
output.show() 

In [69]:
query.awaitTermination()

StreamingQueryException: [STREAM_FAILED] Query [id = afa4f900-222d-4868-93a4-6c0332375acd, runId = ef13b456-f06a-4e43-b34b-a9adec9e16d3] terminated with exception: Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 12) (3e248013a1c0 executor driver): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to purchased_items.json.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:774)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:420)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.$anonfun$executeWrite$2(FileFormatWriter.scala:252)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:93)
	at org.apache.spark.TaskContext.runTaskWithListeners(TaskContext.scala:161)
	at org.apache.spark.scheduler.Task.run(Task.scala:141)
	at org.apache.spark.e

In [ ]:
type(query)